<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/*ubrb_precip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Carli Beisel

Purpose: This script extracts annual precipitation data from Google Earth Engine using the Earth Engine API in Python for the Upper Boise River Basin.

## Import packages/libraries and connect to Google drive

In [ ]:
# Installs geemap package
import subprocess
!pip install geemap

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet') #input and create project on google earth engiine

In [ ]:
!pip install geopandas
import geopandas as gpd
import json
import os
import numpy as np
import pandas as pd
!pip install pycrs
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

## Retrieve Water Year Precipitation for UBRB

In [ ]:
## ------------------------------------------------- ##
##        WATER YEAR PRECIPITATION METRIC            ##
## ------------------------------------------------- ##

# NOTE: wy = Water Year: October 1 - September 30


ubrb = emap.shp_to_ee('/content/drive/MyDrive/Data/GIS Shapefiles/UBRB_shp/UBRB.shp')

years = np.arange(1987,2021)
wy_pr = []

for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-01'), (str(years[i])+'-9-30')) #get image collection for water year
  pr = daymet.select('prcp').map(lambda image: image.clip(ubrb)).sum().set({'system:index': str(years[i])}) #select precip to analyze and sum
  wy_pr.append(pr)

wy_pr = ee.ImageCollection(wy_pr)


out_stats = os.path.join('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
emap.zonal_statistics(wy_pr, ubrb, out_stats, statistics_type='MEAN', scale=1000)

In [ ]:
#---------------------#
#        Clean        #
#---------------------#
#Purpose: reorganize csv file to make manageable

#reload file
data = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')

df = pd.DataFrame(data)
df = df.T #transpose df

# Set the first row as the column names
df.columns = df.iloc[0]
df = df[0:]
df = df.drop(df.index[34:]) #drop the last few lines of data description

# Reset index and rename columns
df = df.reset_index()
df.columns = ['year', 'precip']

# Remove '_prcp' from the 'year' column
df['year'] = df['year'].str.replace('_prcp', '')

print(df)

#save
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv', index=False)


In [ ]:
#---------------------#
#        Scale        #
#---------------------#
mean_precip = df['precip'].mean()
std_dev_precip = df['precip'].std()
df['precip_scaled'] = (df['precip'] - mean_precip) / std_dev_precip ** 2

df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp_scaled.csv', index=False)

In [ ]:
## --------------------------------------------------- ##
##   Regresion Plot: UBRB Water Year Precipitation     ##
## --------------------------------------------------- ##

df = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')

sns.regplot(x=df["year"], y=df["precip"], marker='o', data=df)

plt.title('Upper Boise River Basin Precipitation')
plt.xlabel('Water Year')
plt.ylabel('Precipitation (units)')

##**LBRB precip** : For reference

In [ ]:
# Load the shapefile
lbrb = emap.shp_to_ee('/content/drive/MyDrive/Data/GIS Shapefiles/LBRB_shp/LBRB_file.shp') # converts shapefile to feature in GEE


years = np.arange(1987,2021)
wy_pr = []

for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-01'), (str(years[i])+'-09-30')) #get image collection for entire year
  pr = daymet.select('prcp').map(lambda image: image.clip(lbrb)).sum().set({'system:index': str(years[i])}) #select precip to analyze and sum
  wy_pr.append(pr)

annual_pr = ee.ImageCollection(annual_pr)


out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/UBRB_precip/lbrb_prcp.csv')
emap.zonal_statistics(wy_pr, lbrb, out_stats, statistics_type='MEAN', scale=1000)

In [ ]:
#---------------------#
#        Clean        #
#---------------------#
#Purpose: reorganize csv file to make manageable

#reload file
data = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/lbrb_prcp.csv')

df = pd.DataFrame(data)
df = df.T #transpose df

# Set the first row as the column names
df.columns = df.iloc[0]
df = df[0:]
df = df.drop(df.index[34:]) #drop the last few lines

# Reset the index and rename the columns
df = df.reset_index()
df.columns = ['year', 'precip']

# Remove '_prcp' from the 'year' column
df['year'] = df['year'].str.replace('_prcp', '')

print(df)

#save
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/lbrb_prcp.csv', index=False)

In [ ]:
#---------------------#
#        Scale        #
#---------------------#
mean_precip = df['precip'].mean()
std_dev_precip = df['precip'].std()
df['precip_scaled'] = (df['precip'] - mean_precip) / std_dev_precip ** 2

df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/lbrb_prcp_scaled.csv', index=False)

## Figures

In [ ]:
## --------------------------------------------------- ##
##   Regresion Plot: UBRB Water Year Precipitation     ##
## --------------------------------------------------- ##

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')

# Prepare the data
X = df[['year']].values
y = df['precip'].values

# Create and fit the model
model = LinearRegression()
model.fit(X, y)

# Predict using the model
y_pred = model.predict(X)

# Calculate the R^2 value
r_squared = r2_score(y, y_pred)

# Plot the data and the regression line
sns.regplot(x='year', y='precip', data=df, marker='o', scatter_kws={'color': 'blue'}, line_kws={'color': 'red'})

# Display the R^2 value on the plot
plt.text(0.05, 0.95, f'$R^2 = {r_squared:.2f}$', fontsize=12, color='red', transform=plt.gca().transAxes)

# Set plot labels and title
plt.title('Upper Boise River Basin Precipitation')
plt.xlabel('Water Year')
plt.ylabel('Precipitation (units)')

# Show plot
plt.show()

# Print the R^2 value
print(f'R^2 Coefficient: {r_squared:.2f}')

## Comparison of UBRB & LBRB Water Year Precip

In [ ]:
ubrb = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
lbrb = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/lbrb_prcp.csv')

# Add an identifier column to each DataFrame
ubrb['Dataset'] = 'UBRB'
lbrb['Dataset'] = 'LBRB'

# Combine the datasets into one DataFrame
combined_df = pd.concat([ubrb, lbrb])


plt.figure(figsize=(12, 6))

#specify color
palette = {'UBRB': '#337DFF',  # Blue
           'LBRB': '#EA1612'}  # Red

sns.lineplot(
    x='year',
    y='precip',
    hue='Dataset',
    style='Dataset',
    data=combined_df,
    palette=palette,
    markers=True,  # Enable markers for each line
    dashes={'UBRB': '', 'LBRB': (2, 2)}  # Dash pattern for each dataset
)


plt.title('Comparison of Precipitation in the Upper and Lower Boise River Basin')
plt.xlabel('Year')
plt.ylabel('Precipitation (units)')
plt.legend(title='Dataset')

plt.savefig('/content/drive/MyDrive/Carli Thesis/Figures/Model Figures/UBRB_LBRB_WY_Precip.tiff', format='tiff', dpi=300, bbox_inches='tight')

plt.show()
plt.close()